In [55]:

const tf = require('@tensorflow/tfjs-node') as typeof import('@tensorflow/tfjs-node');



const model = await tf.loadGraphModel('file://./models/lpr-8n_web_model/model.json');


In [2]:
const config = {
    modelPath: './models/lpr-8n.onnx',
    inputWidth: 640,
    inputHeight: 640,
    sourceWidth: 3024,
    sourceHeight: 4032,
    sourceImage: './sample-images/car-with-licenseplate.jpeg'
};

In [3]:
const { JSDOM } = require('jsdom') as typeof import('jsdom');
const { Canvas, createCanvas, Image, ImageData, loadImage } = require('canvas') as typeof import('canvas');
const jo = require('jpeg-autorotate') as typeof import('jpeg-autorotate');




function installDOM() {
  const dom = new JSDOM();
  global.document = dom.window.document;
  // The rest enables DOM image and canvas and is provided by node-canvas
  global.Image = Image;
  global.HTMLCanvasElement = Canvas;
  global.ImageData = ImageData;
  global.HTMLImageElement = Image;
}


installDOM();
let {error, buffer} = await jo.rotate(config.sourceImage, {useExifOrientation: true, quality: 100});
const image = await loadImage(buffer);

console.log('DOM Initialized to allow canvas and image');
  
export { };

DOM Initialized to allow canvas and image


In [5]:
import cv from "@techstark/opencv-js";
const mat = cv.imread(image); // read from img tag
const matC4 = new cv.Mat(); // new image matrix
const matC5 = new cv.Mat(mat.rows, mat.cols, cv.CV_8UC3); // new image matrix
const matC6 = new cv.Mat(mat.rows, mat.cols, cv.CV_8UC3); // new image matrix

mat.channels
cv.cvtColor(mat,matC4, cv.COLOR_BGR2RGBA); // RGBA to BGR  COLOR_BGR2RGBA
cv.resize(matC4, matC5, new cv.Size(config.inputWidth, config.inputHeight)); // resize
cv.cvtColor(matC5,matC6, cv.COLOR_BGRA2RGB)

const input = cv.blobFromImage(
    matC6,
    1 / 255.0, // normalize
    new cv.Size(config.inputWidth, config.inputHeight), // resize to model input size
    new cv.Scalar(0, 0, 0),
    true, // swapRB
    false // crop
); // preprocessing image matrix
console.log('input',input.size());

export {}

input { width: 3, height: 1 }


In [28]:
import fs from "fs";
import cv from "@techstark/opencv-js";

const imageFIle = fs.readFileSync('./sample-images/car-with-licenseplate.jpeg');
const tfImg = tf.node.decodeImage(imageFIle, 3);
const smalImg = tf.image.resizeBilinear(tfImg, [640, 640]);
const tensor = tf.cast(smalImg, 'float32').expandDims(0).div(tf.scalar(255));

//const image = fs.readFileSync('./sample-images/matPad5.jpg');
//let tensor = tf.node.decodeImage(resized, 3);//.expandDims(0);

In [ ]:
import tf from '@tensorflow/tfjs-node';
const output = (await model.predict(tensor)) as tf.Tensor;
console.log('output',output);

const result = output.dataSync();
console.log('shape',output.shape);
console.log('result',JSON.stringify(result));
/*
.executeAsync(tensor).then((result) => {
    (result as any).dataSync();
    console.log('result',JSON.stringify(result));
    //console.log('strides',result.strides[0][1 ])
   // console.log('result',JSON.stringify(result.slice(0,3)));
});
*/
//const result = model.predict(tensor)
//.net.exec()
//const result = model.detect(tensor, { batchSize: 1, verbose: true });

console.log('result',JSON.stringify(result.slice(0,3)));
export {}
const boxes:any[] = [];
for(let i=0;i < output.shape[2];i++){
    const row = result.slice(0,4);
    const scores = result.slice(4);
    for(let j=0;j < output.shape[2];j++){
        const box = result.slice(0,4);
        boxes.push(box);
    }
}
console.log('total boxes',boxes.length);

output Tensor {
  kept: false,
  isDisposedInternal: false,
  shape: [ 1, 5, 8400 ],
  dtype: 'float32',
  size: 42000,
  strides: [ 42000, 8400 ],
  dataId: {},
  id: 6449,
  rankType: '3',
  scopeId: 8570
}
shape [ 1, 5, 8400 ]
result {"0":5.691438674926758,"1":7.597042560577393,"2":21.337371826171875,"3":37.86625671386719,"4":44.585933685302734,"5":44.884803771972656,"6":46.25992202758789,"7":47.896629333496094,"8":52.15796661376953,"9":83.06797790527344,"10":93.00328826904297,"11":100.89215087890625,"12":110.64493560791016,"13":115.85617065429688,"14":119.62401580810547,"15":117.5721435546875,"16":123.6789779663086,"17":138.27516174316406,"18":152.6659698486328,"19":160.96820068359375,"20":161.3265838623047,"21":163.4851531982422,"22":166.16590881347656,"23":174.3184814453125,"24":185.97630310058594,"25":200.4742431640625,"26":215.70675659179688,"27":217.9945526123047,"28":216.50506591796875,"29":218.95791625976562,"30":243.88661193847656,"31":260.60552978515625,"32":263.0889282226

: 

: 

In [60]:
const canvasRef = createCanvas(config.sourceWidth, config.sourceHeight);
const ctx = canvasRef.getContext('2d');

const [boxes, scores, classes] = output.slice(0, 3);
const boxes_data = boxes.dataSync();
const scores_data = scores.dataSync();
const classes_data = classes.dataSync();
const classThreshold = 0.5;

const ratios = [canvasRef.width / config.sourceWidth, canvasRef.height / config.sourceHeight];
const colors = new Colors();

const font = `${Math.max(
  Math.round(Math.max(ctx.canvas.width, ctx.canvas.height) / 40),
  14
)}px Arial`;
ctx.font = font;
ctx.textBaseline = "top";

for (let i = 0; i < scores_data.length; ++i) {
  // filter based on class threshold
  if (scores_data[i] > classThreshold) {
    const klass = labels[classes_data[i]];
    const color = colors.get(classes_data[i]);

    let [x1, y1, x2, y2] = boxes_data.slice(i * 4, (i + 1) * 4);
    x1 *= canvasRef.width * ratios[0];
    x2 *= canvasRef.width * ratios[0];
    y1 *= canvasRef.height * ratios[1];
    y2 *= canvasRef.height * ratios[1];
    const width = x2 - x1;
    const height = y2 - y1;

    const colors = new Colors();

    // draw box.
    ctx.fillStyle = Colors.hexToRgba("FFFFF", 0.2);
    ctx.fillRect(x1, y1, width, height);
    // draw border box.
    ctx.strokeStyle = colors.get(classes_data[i]);
    ctx.lineWidth = Math.max(Math.min(ctx.canvas.width, ctx.canvas.height) / 200, 2.5);
    ctx.strokeRect(x1, y1, width, height);

    // Draw the label background.
    ctx.fillStyle = color;
    const textWidth = ctx.measureText(klass + " - " + score + "%").width;
    const textHeight = parseInt(font, 10); // base 10
    const yText = y1 - (textHeight + ctx.lineWidth);
    ctx.fillRect(
      x1 - 1,
      yText < 0 ? 0 : yText, // handle overflow label box
      textWidth + ctx.lineWidth,
      textHeight + ctx.lineWidth
    );

    // Draw labels
    ctx.fillStyle = "#ffffff";
    ctx.fillText(klass + " - " + score + "%", x1 - 1, yText < 0 ? 0 : yText);

  }
}

class Colors {
  // ultralytics color palette https://ultralytics.com/
  public readonly palette: string[];
  public readonly n: number;
  constructor() {
    this.palette = [
      "#FF3838",
      "#FF9D97",
      "#FF701F",
      "#FFB21D",
      "#CFD231",
      "#48F90A",
      "#92CC17",
      "#3DDB86",
      "#1A9334",
      "#00D4BB",
      "#2C99A8",
      "#00C2FF",
      "#344593",
      "#6473FF",
      "#0018EC",
      "#8438FF",
      "#520085",
      "#CB38FF",
      "#FF95C8",
      "#FF37C7",
    ];
    this.n = this.palette.length;
  }

  get = (i) => this.palette[Math.floor(i) % this.n];

  static hexToRgba = (hex, alpha) => {
    var result = /^#?([a-f\d]{2})([a-f\d]{2})([a-f\d]{2})$/i.exec(hex);
    return result
      ? `rgba(${[parseInt(result[1], 16), parseInt(result[2], 16), parseInt(result[3], 16)].join(
          ", "
        )}, ${alpha})`
      : null;
  };
}

4:7 - Type 'Tensor<Rank>' must have a '[Symbol.iterator]()' method that returns an iterator.
11:20 - Class 'Colors' used before its declaration.
23:19 - Cannot find name 'labels'.
24:19 - Block-scoped variable 'colors' used before its declaration.
34:24 - Class 'Colors' used before its declaration.
37:21 - Class 'Colors' used before its declaration.
46:55 - Cannot find name 'score'. Did you mean 'scores'?
58:34 - Cannot find name 'score'. Did you mean 'scores'?
